In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#to display all rows columns 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [2]:
ipl_df = pd.read_csv('all_matches.csv')

In [3]:
ipl_df.batting_team.unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Deccan Chargers', 'Mumbai Indians',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans'], dtype=object)

In [4]:
ipl_df = ipl_df[(ipl_df.innings ==1) | (ipl_df.innings ==2)]

In [5]:
ipl_df['total_runs'] = ipl_df.apply(lambda x:x['runs_off_bat']+x['extras'],axis =1)

In [6]:
ipl_df['isOut'] = ipl_df['player_dismissed'].apply(lambda x: 1 if type(x) == type('str') else 0)

In [7]:
# Win prediction
# match_id = 1370353

t1= 'Gujarat Titans'
t2 = 'Chennai Super Kings'

In [8]:
matchID = 1370353

In [9]:
mdf = ipl_df[ipl_df.match_id == matchID]

In [10]:
outcomes = [0,1,2,3,4,6,'w']

In [11]:
ipl_df[ipl_df.batting_team==t1].total_runs.value_counts()

1    1796
0    1209
4    514 
2    277 
6    202 
5    13  
3    9   
7    1   
Name: total_runs, dtype: int64

In [12]:
ipl_df[ipl_df.batting_team==t1].isOut.sum()

168

In [13]:
t1_outcomes_count = ipl_df[ipl_df.batting_team==t1].total_runs.value_counts()
t2_outcomes_count = ipl_df[ipl_df.batting_team==t2].total_runs.value_counts()

In [14]:
t1_outs = ipl_df[ipl_df.batting_team==t1].isOut.sum()
t2_outs = ipl_df[ipl_df.batting_team==t2].isOut.sum()

In [15]:
t1_outcomes = []
t2_outcomes = []

for outcome in outcomes:
    if outcome!= 'w':
        t1_outcomes.append(t1_outcomes_count[outcome])
        t2_outcomes.append(t2_outcomes_count[outcome])
    else:
        t1_outcomes.append(t1_outs)
        t2_outcomes.append(t2_outs)

In [16]:
t1_outcomes ,t2_outcomes 

([1209, 1796, 277, 9, 514, 202, 168],
 [9171, 11334, 1839, 106, 3029, 1392, 1166])

In [17]:
t1_pb_outcomes = [(i/sum(t1_outcomes)) for i in t1_outcomes]
t2_pb_outcomes = [(i/sum(t2_outcomes)) for i in t2_outcomes]

In [18]:
t1_pb_outcomes,t2_pb_outcomes

([0.2895808383233533,
  0.43017964071856285,
  0.06634730538922155,
  0.002155688622754491,
  0.12311377245508982,
  0.04838323353293413,
  0.040239520958083835],
 [0.32710347041409565,
  0.40425152477083853,
  0.06559189642258444,
  0.003780718336483932,
  0.10803580982273424,
  0.04964867853193994,
  0.04158790170132325])

# Fetch probability values:

In [19]:
t1_pb_is = np.cumsum(t1_pb_outcomes)
t2_pb_is = np.cumsum(t2_pb_outcomes)

In [20]:
t1_pb_is

array([0.28958084, 0.71976048, 0.78610778, 0.78826347, 0.91137725,
       0.95976048, 1.        ])

In [21]:
t2_pb_is

array([0.32710347, 0.731355  , 0.79694689, 0.80072761, 0.90876342,
       0.9584121 , 1.        ])

In [22]:
def predict_runs(target, current_score, current_wickets, current_overs):
    
    i1p_0 = t1_pb_is[0]
    i1p_1 = t1_pb_is[1]
    i1p_2 = t1_pb_is[2]
    i1p_3 = t1_pb_is[3]
    i1p_4 = t1_pb_is[4]
    i1p_6 = t1_pb_is[5]
    i1p_w = t1_pb_is[6]

    i2p_0 = t2_pb_is[0]
    i2p_1 = t2_pb_is[1]
    i2p_2 = t2_pb_is[2]
    i2p_3 = t2_pb_is[3]
    i2p_4 = t2_pb_is[4]
    i2p_6 = t2_pb_is[5]
    i2p_w = t2_pb_is[6]

    pred_runs = current_score
    pred_wks = current_wickets
    over_ball = current_overs
    
#     over_no = int(str(over_ball).split('.')[0])
#     ball_no = int(str(over_ball).split('.')[1])
    
    leftover_balls = 120-current_overs*6

    for i in range(leftover_balls):
        r_value = np.random.random()

        if r_value <= i2p_0:
            pred_runs += 0
        elif r_value <= i2p_1:
            pred_runs += 1
        elif r_value <= i2p_2:
            pred_runs += 2
        elif r_value <= i2p_3:
            pred_runs += 3
        elif r_value <= i2p_4:
            pred_runs += 4
        elif r_value <= i2p_6:
            pred_runs += 6
        else:
            pred_runs += 0
            pred_wks += 1
            if pred_wks == 10:
                break
        if pred_runs > target:
            break
        # print('pred_runs: ', pred_runs)
        # print('pred_wks: ', pred_wks)
    
    return pred_runs

In [23]:
# predict_runs(target, current_score, current_wickets, current_overs)
predict_runs(214, 0, 0, 0)

160

# Winner function:

In [24]:
def get_win(pred_runs, target):
    if pred_runs > target:
        return 'win'
    elif pred_runs == target:
        return 'tie'
    else:
        return 'lose'

In [25]:
# runs, wickets, overs, 
# win - 1st ing score

# GT - 214/4

target = 214

current_score = 117
current_wickets = 3
current_overs = 11

iter_count = 100

runs_ls = []
results_ls = []

win_count = 0
tie_count = 0
lose_count = 0

for i in range(iter_count):
    pred_runs = predict_runs(target, current_score, current_wickets, current_overs)
    runs_ls.append(pred_runs)
    result_pred = get_win(pred_runs, target)
    results_ls.append(result_pred)

    if result_pred == 'win':
        win_count += 1
    elif result_pred == 'tie':
        tie_count += 1
    else:
        lose_count +=1

In [26]:
win_count, tie_count, lose_count

(3, 1, 96)

# Find out Runs at a required stage:

In [27]:
# find out runs:
def find_runs(current_score, target, current_wickets, at_overs):
    runs_ls = []
    results_ls = []

    req_runs = []
    win_ls = []

    for i in range(current_score, target + 1):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
            pred_runs = predict_runs(target, i, current_wickets, at_overs)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count +=1

            win_ls.append(win_count)
            req_runs.append(i)
            # print('runs: ', i, ' win%: ', win_count)

    required_runs = current_score
    for i in range(len(req_runs)):
        if win_ls[i] >= 50:
            required_runs = req_runs[i]
            # print('Runs to be: ', req_runs[i])
            break

    return required_runs


In [28]:
# find_runs(current_score, target, current_wickets, at_overs)
find_runs(117, 214, 3, 11)

144

# Find out Wickets they can afford to lose:

In [29]:

def find_wickets(current_score, target, current_wickets, at_overs):

#     find_runs(current_score, target, current_wickets, at_overs)
    req_runs = find_runs(current_score, target, current_wickets, at_overs)

    runs_ls = []
    results_ls = []

    req_wks = []
    win_ls = []

    for i in range(current_wickets, 10):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
#             pred_runs = predict_runs(target, req_runs, i, at_overs)
            pred_runs = predict_runs(target, current_score, i, at_overs)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count +=1

        win_ls.append(win_count)
        req_wks.append(i)
        #print('wickets: ', i, ' win%: ', win_count)

    req_wicket_value = current_wickets
    
    for i in range(len(req_wks)):
        if (win_ls[i] < 45)  :
            req_wicket_value = req_wks[i]
            break

    return req_wicket_value

In [30]:
# wickets they can afford to lose:

# find_wickets(current_score, target, current_wickets, at_overs)
find_wickets(117, 167, 3, 11)

9

# Function: Interactive chart

In [31]:
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

def find_runs_wickets(current_wks, at_overs, target_score):
    plt.figure(figsize = (16, 6))
    # x = np.linspace(-10, 10, num=1000)
    x = np.array(list(range(21)))
    req_value = find_runs(117, target_score, current_wks, at_overs)
    req_wk_value = find_wickets(117, target_score, current_wks, at_overs)

    if at_overs == 11:
        req_value = 117
        req_wk_value = 3

    
    # print(req_value)
    y = np.array([req_value for i in range(21)])

    # plt.plot(x, current_overs * x + target_score)
    # plt.plot(x, y)
    plt.scatter(at_overs, req_value, s = 1200, color = 'red')
    plt.axhline(target_score, ls = '--', color = 'blue')
    plt.text( 1, target_score + 10, 'Target Score :' + str(target_score) , color = 'darkblue', fontsize = 13)
    plt.text( at_overs, req_value, str(req_value) + '/' + str(req_wk_value), color = 'white', fontsize = 12,  horizontalalignment='center', verticalalignment='center')
    plt.text(at_overs, req_value - 30, 'CSK has to be at ' + str(req_value) + '/' +  str(req_wk_value) + ' after ' + str(at_overs) + ' ov', horizontalalignment='center')
    plt.ylim(50, target_score + 50)
    plt.xticks(x)
    plt.title('Where should CSK be?', fontsize = 20)
    plt.xlabel('Overs')
    plt.ylabel('Score')
    plt.show()

# x=widgets.IntSlider(min=-10, max=30, step=1, value=10)

# find_wickets(current_score, target, current_wickets, at_overs)
# find_wickets(117, 214, 3, 11)


# Interactive Visualization:

In [32]:
print('current_score = CSK: 117/3 (11 overs)')
print('')

interactive_plot = interactive(find_runs_wickets, current_wks = widgets.IntSlider(min=1, max=10, step=1, value=3),  at_overs=widgets.IntSlider(min=10, max=20, step=1, value=11), target_score = widgets.IntSlider(min=0, max=250, step=1, value=214))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

current_score = CSK: 117/3 (11 overs)



interactive(children=(IntSlider(value=3, description='current_wks', max=10, min=1), IntSlider(value=11, descri…

In [33]:
# Function: Interactive chart

%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

def find_runs_wickets(current_wks, at_overs, target_score):
    plt.figure(figsize = (16, 6))
    # x = np.linspace(-10, 10, num=1000)
    x = np.array(list(range(21)))
    req_value = find_runs(117, target_score, current_wks, at_overs)
    req_wk_value = find_wickets(117, target_score, current_wks, at_overs)

    if at_overs == 11:
        req_value = 117
        req_wk_value = 3

    
    # print(req_value)
    y = np.array([req_value for i in range(21)])

    # plt.plot(x, current_overs * x + target_score)
    # plt.plot(x, y)
    plt.scatter(at_overs, req_value, s = 1200, color = 'red')
    plt.axhline(target_score, ls = '--', color = 'blue')
    plt.text( 1, target_score + 10, 'Target Score :' + str(target_score) , color = 'darkblue', fontsize = 13)
    plt.text( at_overs, req_value, str(req_value) + '/' + str(req_wk_value), color = 'white', fontsize = 12,  horizontalalignment='center', verticalalignment='center')
    plt.text(at_overs, req_value - 30, 'CSK has to be at ' + str(req_value) + '/' +  str(req_wk_value) + ' after ' + str(at_overs) + ' ov', horizontalalignment='center')
    plt.ylim(50, target_score + 50)
    plt.xticks(x)
    plt.title('Where should CSK be?', fontsize = 20)
    plt.xlabel('Overs')
    plt.ylabel('Score')
    plt.show()

# x=widgets.IntSlider(min=-10, max=30, step=1, value=10)

# find_wickets(current_score, target, current_wickets, at_overs)
# find_wickets(117, 214, 3, 11)


# Interactive Visualization:

print('current_score = CSK: 117/3 (11 overs)')
print('')

interactive_plot = interactive(find_runs_wickets, current_wks = widgets.IntSlider(min=1, max=10, step=1, value=3),  at_overs=widgets.IntSlider(min=10, max=20, step=1, value=11), target_score = widgets.IntSlider(min=0, max=250, step=1, value=214))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

current_score = CSK: 117/3 (11 overs)



interactive(children=(IntSlider(value=3, description='current_wks', max=10, min=1), IntSlider(value=11, descri…